In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
labels = os.listdir('../input/natural_images/')
print(labels)

In [ ]:
from IPython.display import Image, display
num = []
for label in labels:
    path = '../input/natural_images/{0}/'.format(label)
    folder_data = os.listdir(path)
    k = 0
    print('\n', label.upper())
    for image_path in folder_data:
        if k < 5:
            display(Image(path+image_path))
        k = k+1
    num.append(k)
    print('there are ', k,' images in ', label, 'class')

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (8,8))
plt.bar(labels, num)
plt.title('NUMBER OF IMAGES CONTAINED IN EACH CLASS')
plt.xlabel('classes')
plt.ylabel('count')
plt.show()

In [ ]:
x_data =[]
y_data = []
import cv2
for label in labels:
    path = '../input/natural_images/{0}/'.format(label)
    folder_data = os.listdir(path)
    for image_path in folder_data:
        image = cv2.imread(path+image_path)
        image_resized = cv2.resize(image, (32,32))
        x_data.append(np.array(image_resized))
        y_data.append(label)      

In [ ]:
x_data = np.array(x_data)
y_data = np.array(y_data)
print('the shape of X is: ', x_data.shape, 'and that of Y is: ', y_data.shape)

In [ ]:
#stadardizing the input data
x_data = x_data.astype('float32')/255

In [ ]:
#converting the y_data into categorical:
from sklearn.preprocessing import LabelEncoder
y_encoded = LabelEncoder().fit_transform(y_data)
from keras.utils import to_categorical
y_categorical = to_categorical(y_encoded)

In [ ]:
#lets shuffle all the data we have:
r = np.arange(x_data.shape[0])
np.random.seed(42)
np.random.shuffle(r)
X = x_data[r]
Y = y_categorical[r]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33)

In [ ]:
#structuring the CNN model
from keras import models, layers
model = models.Sequential()
model.add(layers.Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(layers.MaxPool2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2, 2)))
model.add(layers.Dropout(rate=0.25))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(8, activation='softmax'))

In [ ]:
#let's compile the model
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])

In [ ]:
#fitting the model
history = model.fit(X_train, Y_train, epochs=25, validation_split=0.2)

In [ ]:
#Display of the accuracy and the loss values
import matplotlib.pyplot as plt

plt.figure(figsize = (8,8))
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss/accuracy')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
from keras.utils import plot_model
plot_model(model)

In [ ]:
#from IPython.display import Image
#import cv2
#import numpy as np
#from keras.preprocessing import image
#img=Image("../input/natural_images/airplane/airplane_0000.jpg")
#img_resized=[cv2.resize(img, (32,32), interpolation=cv2.INTER_LINEAR)]
#img_std=img_resized.astype('float32')/255
#Y_pred = model.predict_classes(img_std)
#Y_pred

#test_image = cv2.resize(cv2.imread("../input/natural_images/airplane/airplane_0000.jpg", cv2.IMREAD_GRAYSCALE),(32,32))
#test_image=cv2.imread("../input/natural_images/airplane/airplane_0000.jpg", cv2.IMREAD_GRAYSCALE)
#print(test_image)
 #test_image = np.array(test_image).reshape(1, 32, 32, 3)
 #test_image = np.expand_dims(test_image, axis=0)

 #Y_pred = model.predict_classes(test_image)

#test_image = image.img_to_array(test_image)
#test_image = np.expand_dims(test_image, axis=0)
#print(test_image.shape)
#result = model.predict(test_image)
#result
#prediction = model.predict({'input': test_image })




In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('../input/natural_images/airplane/airplane_0018.jpg',target_size = (32,32))
#test_image = image.load_img('../input/catimage/cat.jpeg',target_size = (32,32))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = model.predict(test_image)
result


In [ ]:
import pickle
print("[INFO] Saving model...")
pickle.dump(model,open('cnn_model.pkl', 'wb'))

In [ ]:
ls

In [ ]:
from IPython.display import FileLink
FileLink(r'cnn_model.pkl')

In [ ]:
ls -s

In [ ]:
import tensorflow as tf

model.save('cnnmodel.h5')
model=tf.keras.models.load_model("cnnmodel.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)  


In [ ]:
from IPython.display import FileLink
FileLink(r'converted_model.tflite')

In [ ]:
labels = '\n'.join(labels)
with open('labels.txt', 'w') as f:
    f.write(labels)

In [ ]:
from IPython.display import FileLink
FileLink(r'labels.txt')

In [ ]:
#import tensorflow as tf

#from tensorflow import lite


# saving your deep learning model
#model.save('cnnmodel.h5')
#from tensorflow.contrib import lite


#converter = lite.TFLiteConverter.from_keras_model('cnnmodel.h5')
#tflite_model = converter.convert()
# Convert the model.
#converter = tf.lite.TFLiteConverter.from_keras_model(model)
#tflite_model = converter.convert()
#open("cnnmodel1.tflite", "wb").write(tflite_model)

In [ ]:
ls

In [ ]:
from IPython.display import FileLink
FileLink(r'cnnmodel.h5')

In [ ]:
#converting over Y test to actual labels.
#Y_test = np.argmax(Y_test, axis = 1)

In [ ]:
#from sklearn.metrics import accuracy_score
#print('the accuracy obtained on the test set is:', accuracy_score(Y_pred,Y_test))

In [ ]:
#from sklearn.metrics import classification_report
#print(classification_report(Y_test, Y_pred))